In [ ]:
# optimize pp and pca_number
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

import matplotlib as mpl
mpl.rc("figure", dpi=200)

In [ ]:
%time 
# load the data
from natto import input 
from natto import process
from ubergauss import tools
import numpy as np


def loadandgo(n,pp='natto'):
    data = input.load100(n,path = '/home/ubuntu/repos/natto/natto/data',remove_unlabeled=True)
    gs= process.annotate_genescores(data, mingenes = 200, selector = pp)
    #gs.X = np.expm1(gs.X)
    return gs 

#annotated = 
from ubergauss import tools
names = input.get41names()#[:6]
dists = tools.loadfile('dist41.dmp')
#np.fill_diagonal(dists,-999)

In [ ]:
from natto.optimize.x40 import assign 
from sklearn.metrics import adjusted_rand_score
from functools import partial
import scanpy as sc
from natto.tools import compare

def score(arg):
    a1,a2 = arg
    '''
    propagate labels
    score 
    '''
    scores = []
    for i in range(3):
        minobs = min(a1.X.shape[0], a2.X.shape[0])
        n_obs = min(1500, minobs)
        b1 = sc.pp.subsample(a1, copy=True, n_obs = n_obs)
        b2 = sc.pp.subsample(a2, copy=True, n_obs = n_obs)
        m=process.merge([b1,b2],umaps=[], pca=20, sortfield=1) # sorting is important for hungonly :) 

        defuse_and_score_fixed_neighbors = partial(assign.diffuseandscore, neighbors = 80)
        
        scorez = [ f(m.data[0].obs['true'],
                                       m.data[1].obs['true'],
                                       m.projections[1]) for f in 
                        [defuse_and_score_fixed_neighbors,
                         compare.score1nn,compare.hungonly, compare.labelprobonly]]
        
        scores.append(scorez)
        
    return np.mean(scores, axis = 0)
    
scrs = []
for pp in "natto seurat_v3 cell_ranger seurat".split()[:1]:
    f= partial(loadandgo, pp=pp)
    adataL = tools.xmap(f,names)
    scores = tools.xmap(score, ( (adataL[i], adataL[np.argmax(dists[i])])  for i in range(len(names)) ))
    scrs.append(np.mean(scores, axis  = 0))

print(scrs) 

'''
ours: 15 # was at 50 
1nn: 23 
hungarianonly 15
labelpropagation: 0 
'''

In [ ]:
assign.opti()